In [61]:
import os
import openai
import pandas as pd
import tiktoken  # for counting tokens
import ast  # for converting embeddings saved as strings back to arrays
from scipy import spatial
import pretty_errors
from preprocess import individual_preprocess
from displayfunction import display
from dotenv import load_dotenv

In [62]:

load_dotenv('.env')
openai.api_key = os.getenv("OPENAI_API_KEY")
SAVE_PATH = os.getenv("SAVE_PATH")

# models
EMBEDDING_MODEL = "text-embedding-ada-002"
#GPT_MODEL = "gpt-3.5-turbo"
GPT_MODEL = "gpt-4"

"""
Load the embedded file
"""

embeddings_path = SAVE_PATH + "/jobs_test5.csv"

df = pd.read_csv(embeddings_path)

# convert embeddings from CSV str type back to list type
df['embedding'] = df['embedding'].apply(ast.literal_eval)

"""
## 2. Search

Now we'll define a search function that:
- Takes a user query and a dataframe with text & embedding columns
- Embeds the user query with the OpenAI API
- Uses distance between query embedding and text embeddings to rank the texts
- Returns two lists:
    - The top N texts, ranked by relevance
    - Their corresponding relevance scores

"""

In [63]:

# search function
def ids_ranked_by_relatedness(
    query: str,
    df: pd.DataFrame,
    relatedness_fn=lambda x, y: 1 - spatial.distance.cosine(x, y),
    #Modify this to get more jobs
    top_n: int = 15
) -> tuple[list[str], list[float]]:

    """Returns a list of strings and relatednesses, sorted from most related to least."""
    
    query_preprocess = individual_preprocess(query)
    query_embedding_response = openai.Embedding.create(
        model=EMBEDDING_MODEL,
        input=query_preprocess,
    )
    #This is the query (e.g., Data Engineer) that the model will find relatednessnes
    query_embedding = query_embedding_response["data"][0]["embedding"]
    ids_and_relatednesses = [
        (row["id"], relatedness_fn(query_embedding, row["embedding"]))
        for i, row in df.iterrows()
    ]
    ids_and_relatednesses.sort(key=lambda x: x[1], reverse=True)
    ids, relatednesses = zip(*ids_and_relatednesses)
    return ids[:top_n], relatednesses[:top_n]


In [64]:
#tiktoken function -> to count tokens
def num_tokens(text: str, model: str = GPT_MODEL) -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

In [65]:
introduction_prompt = """

To assist you, here are the job IDs and the respective job descriptions ranked by relatedness to the user's specifications.
Review these jobs and output the ID(s) of the job(s) that meet the following criteria:
1. The job(s) contain the user's requirements; or
2. The job(s) contain both "requirements" and "preferences". In this case, called them: "Dream Jobs"

If no job meets all “requirements”, respond with "I am sorry, I do not have your dream job just yet."

"""

In [66]:
def query_message(
    query: str,
    df: pd.DataFrame,
    model: str,
    token_budget: int
) -> str:
    """Return a message for GPT, with relevant source texts pulled from a dataframe."""
    ids, relatednesses = ids_ranked_by_relatedness(query, df)
    #Basically giving the most relevant IDs from the previous function
    introduction = introduction_prompt
    question = f"\n\Requirements and Preferences of the user: {query}"
    message = introduction
    for id in ids:
        #Get the text for GPT to answer the question
        job_description = df[df['id'] == id]['text'].values[0] 
        
        # Add job descriptions to the message along with job ID
        next_id = f'\n\nJob\'s ID:\n"""\n{id}\n"""\nJob Description:\n"""\n{job_description}\n"""'
        if (
            num_tokens(message + next_id + question, model=model)
            > token_budget
        ):
            break
        else:
            message += next_id
    return message + question


In [67]:
system_prompt="""

You are DreamJobAI, an expert in job searching. 
Users will share the “requirements” and “preferences” of their dream job.
You are going to be provided with jobs ranked by relatedness to the user's “requirements” and “preferences”, use those jobs to complete your task.
*Your task is to find jobs which either meet the user's "requirements" or the user's "requirements" and “preferences”.* 
"""


In [68]:
user_reminder = """

*Your task is to find jobs which either meet the user's "requirements" or the user's "requirements" and “preferences”.* 

"""

In [69]:
user_example_1= """

REQUIREMENTS: A job with Python. PREFERENCES: Data Engineer, Machine Learning """

assistant_example_1= """ 

These jobs meet your requirements, because they require someone fluent in Python:

Job's ID: 1
Job's ID: 2
Job's ID: 3
Job's ID: 4
Job's ID: 5

I found your Dream Job! It requires Python (requirement) and it is in Machine Learning (preferences):

Job's ID: 3

"""



In [73]:
def ask(
    #This query is your question, only parameter to fill in function
    query: str,
    df: pd.DataFrame = df,
    model: str = GPT_MODEL,
    token_budget: int = 4096,
    print_messages: bool = True,
) -> str:
    #Answers a query using GPT and a dataframe of relevant texts and embeddings.
    message = query_message(query, df, model=model, token_budget=token_budget)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": message},
        {"role": "system", "name":"example_user", "content": user_example_1},
        {"role": "system", "name": "example_assistant", "content": assistant_example_1},
        {"role": "user", "content": user_reminder}
    ]
    if print_messages:
        print(messages)
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0
    )
    response_message = response["choices"][0]["message"]["content"]
    #Tokens
    print("'\nTOTAL TOKENS USED:\n'", response['usage']['total_tokens'])
    
    #relatednesses
    ids, relatednesses = ids_ranked_by_relatedness(query=query, df=df)
    print(f"\nTHE IDs ARE RANKED BY RELEVANCE:\n")
    for id, relatedness in zip(ids, relatednesses):
        print(f"ID: {id} has the following {relatedness=:.3f}")
    return response_message

In [75]:
print(ask('REQUIREMENTS: Ciudad de México. PREFERENCES: Python, Java, PHP'))

[{'role': 'system', 'content': '\n\nYou are DreamJobAI, an expert in job searching. \nUsers will share the “requirements” and “preferences” of their dream job.\nYou are going to be provided with jobs ranked by relatedness to the user\'s “requirements” and “preferences”, use those jobs to complete your task.\n*Your task is to find jobs which either meet the user\'s "requirements" or the user\'s "requirements" and “preferences”.* \n'}, {'role': 'user', 'content': '\n\nTo assist you, here are the job IDs and the respective job descriptions ranked by relatedness to the user\'s specifications.\nReview these jobs and output the ID(s) of the job(s) that meet the following criteria:\n1. The job(s) contain the user\'s requirements; or\n2. The job(s) contain both "requirements" and "preferences". In this case, called them: "Dream Jobs"\n\nIf no job meets all “requirements”, respond with "I am sorry, I do not have your dream job just yet."\n\n\n\nJob\'s ID:\n"""\n3qfbh5tM\n"""\nJob Description:\n

In [ ]:
#print(ask('REQUIREMENTS: Python developer. PREFERENCES: Machine Learning or Data Engineer'))